In [1]:
%pylab inline
import re
import math
import string
from collections import Counter
from __future__ import division
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
TEXT = open('big.txt').read() #read all the words from big.txt
len(TEXT)

6488665

In [3]:
#we use re package to filter out unnecessary characters, tokenize the words and convert it to lower case
def tokens(text):
    return re.findall('[a-z]+', text.lower()) 

In [4]:
tokens('This is: A test, 1, 2, 3, this is.')

['this', 'is', 'a', 'test', 'this', 'is']

In [5]:
WORDS = tokens(TEXT)
len(WORDS)

1105285

In [6]:
print(WORDS[:10]) #print first few words

['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'adventures', 'of', 'sherlock', 'holmes']


In [7]:
def sample(bag, n=10):
    return ' '.join(random.choice(bag) for _ in range(n))

In [8]:
sample(WORDS)

'which tearing sister of the previously angles heat home he'

In [9]:
Counter(tokens('Is this a test? It is a test!')) #to make up something like a dictionary to store the number of occourences of every word

Counter({'a': 2, 'is': 2, 'it': 1, 'test': 2, 'this': 1})

In [10]:
COUNTS = Counter(WORDS)

print(COUNTS.most_common(10))

[('the', 80030), ('of', 40025), ('and', 38313), ('to', 28766), ('in', 22050), ('a', 21155), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]


In [11]:
for w in tokens('the rare and neverbeforeseen words'):
    print(w,COUNTS[w])

the 80030
rare 83
and 38313
neverbeforeseen 0
words 460


### Spell Checker
        


In [12]:
def correct(word):
    #Generating all the words with edit distance of 0, 1 & 2
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    #print(candidates)
    #print(COUNTS.get)
    return max(candidates, key=COUNTS.get)

In [13]:
def known(words):
    #Return the subset of words that are actually in the dictionary."
    return {w for w in words if w in COUNTS}

def edits0(word): 
    return {word}

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

Now for `edits1(word)`: the set of candidate words that are one edit away. For example, given `"wird"`, this would include `"weird"` (inserting an `e`) and `"word"` (replacing a `i` with a `o`), and also `"iwrd"` (transposing `w` and `i`; then `known` can be used to filter this out of the set of final candidates). How could we get them?  One way is to *split* the original word in all possible places, each split forming a *pair* of words, `(a, b)`, before and after the place, and at each place, either delete, transpose, replace, or insert a letter:

<table>
  <tr><td> pairs: <td><tt> Ø+wird <td><tt> w+ird <td><tt> wi+rd <td><tt>wir+d<td><tt>wird+Ø<td><i>Notes:</i><tt> (<i>a</i>, <i>b</i>)</tt> pair</i>
  <tr><td> deletions: <td><tt>Ø+ird<td><tt> w+rd<td><tt> wi+d<td><tt> wir+Ø<td><td><i>Delete first char of b</i>
  <tr><td> transpositions: <td><tt>Ø+iwrd<td><tt> w+rid<td><tt> wi+dr</tt><td><td><td><i>Swap first two chars of b
  <tr><td> replacements: <td><tt>Ø+?ird<td><tt> w+?rd<td><tt> wi+?d<td><tt> wir+?</tt><td><td><i>Replace char at start of b
  <tr><td> insertions: <td><tt>Ø+?+wird<td><tt> w+?+ird<td><tt> wi+?+rd<td><tt> wir+?+d<td><tt> wird+?+Ø</tt><td><i>Insert char between a and b
</table>

In [14]:
def edits1(word):
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [15]:
splits('wird')

[('', 'wird'), ('w', 'ird'), ('wi', 'rd'), ('wir', 'd'), ('wird', '')]

In [16]:
print(edits0('wird'))

{'wird'}


In [17]:
print(edits1('wird'))

{'wsrd', 'wpird', 'widd', 'wurd', 'wirdw', 'wnird', 'wtrd', 'wiro', 'wiurd', 'awird', 'wikd', 'wixd', 'wkird', 'wirxd', 'wsird', 'rird', 'bwird', 'wqird', 'wirw', 'wiud', 'gwird', 'woird', 'wrird', 'wirdn', 'wicrd', 'wrrd', 'dwird', 'wimrd', 'wiod', 'wxrd', 'wbird', 'wfrd', 'wigrd', 'wird', 'wirqd', 'wirgd', 'wirdz', 'wirf', 'bird', 'mwird', 'wirdq', 'iwird', 'hird', 'pwird', 'wiprd', 'waird', 'wyird', 'wifrd', 'wirdl', 'wlrd', 'wiid', 'pird', 'ewird', 'nird', 'wiri', 'wirm', 'wxird', 'wizrd', 'wjird', 'wrd', 'uird', 'wprd', 'wirdr', 'yird', 'whird', 'wkrd', 'wifd', 'wirq', 'wrid', 'wirr', 'wbrd', 'wirdc', 'nwird', 'jird', 'wgrd', 'wirn', 'wirdy', 'wijrd', 'wirv', 'kird', 'wirmd', 'wirad', 'dird', 'eird', 'wire', 'wirdb', 'wirud', 'twird', 'wirhd', 'wirdh', 'iird', 'vird', 'wcrd', 'wierd', 'wisd', 'kwird', 'swird', 'wcird', 'wiyrd', 'wirld', 'wirdk', 'wuird', 'lwird', 'wisrd', 'werd', 'wirkd', 'owird', 'wivd', 'wyrd', 'weird', 'wicd', 'wied', 'wirh', 'wirk', 'wirdi', 'wfird', 'xwird', 

In [18]:
print(len(edits2('wird')))

24254


In [19]:
list(map(correct, tokens('Speling errurs in somethink. Whutever; unusuel misteakes everyware?')))

['spelling',
 'errors',
 'in',
 'something',
 'whatever',
 'unusual',
 'mistakes',
 'everywhere']

In [20]:
test_df = pd.read_csv('test.csv')
test_df.head(5)

,ID,WRONG
0,0,contenpted
1,1,begining
2,2,problam
3,3,dirven
4,4,exstacy


In [32]:
test_submit_df = pd.read_csv('test_submit_Upload1.csv')
test_submit_df.head(5)

,ID,CORRECT
0,0,contented
1,1,beginning
2,2,problem
3,3,driven
4,4,ecstasy


In [36]:
error_counter =0
error_clash =[]
#COUNTS_dict = dict(COUNTS)
for key in test_df['WRONG']:
    for key2 in test_submit_df['CORRECT']:
        if(key2 == key):
            error_counter += 1
            error_clash.append(key)
            #print(list(map(correct, tokens(key))))
        
print(error_counter)
print(error_clash)

47
['transportibility', 'miniscule', 'stanerdizing', 'diagrammaticaally', 'hierachial', 'hierachial', 'planed', 'addresable', 'hierachial', 'hierachial', 'adabtable', 'latter', 'graphicaly', 'thermawere', 'orentated', 'unresloved', 'unequaled', 'avaiblity', 'subtrcat', 'imidatly', 'exponentualy', 'nessisitates', 'unessasarily', 'unavailble', 'equaled', 'preffeson', 'embelishing', 'similar', 'forth', 'pere', 'necasery', 'unessessay', 'disaggreagte', 'et', 'chose', 'advice', 'accessability', 'encompasing', 'conditining', 'sucssuful', 'humor', 'perametres', 'dissapoiting', 'interogationg', 'drasticaly', 'ineffiect', 'citisum']


In [22]:
test_df.drop('ID', axis=1)
test_list = test_df['WRONG'].tolist()
#print(test_list)

Can we make the output prettier than that?

In [23]:
def correct_text(text):
    return re.sub('[a-zA-Z]+', correct_match, text)

def correct_match(match):
    word = match.group()
    return case_of(word)(correct(word.lower()))

def case_of(text):
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

In [24]:
correct_list =[]
for text in test_list:
    correct_list.append(correct_text(text))
    
#print(correct_list)

In [25]:
import csv

csvfile = "sample_test_submit.csv"

#Assuming res is a flat list
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in correct_list:
        writer.writerow([val])  

In [26]:
correct_text('Speling Errurs IN somethink. Whutever; unusuel misteakes?')

'Spelling Errors IN something. Whatever; unusual mistakes?'

In [27]:
correct_text('Audiance sayzs: tumblr ...')

'Audience says: tumbler ...'

In [28]:
def pdist(counter):
    "Make a probability distribution, given evidence from a Counter."
    N = sum(list(counter.values()))
    return lambda x: counter[x]/N

P = pdist(COUNTS)

In [29]:
for w in tokens('"The" is most common word in English'):
    print(P(w), w)

0.0724066643445 the
0.00884296810325 is
0.000821507574969 most
0.00025966153526 common
0.000269613719538 word
0.0199496057578 in
0.000190900989338 english


In [30]:
def Pwords(words):
    return product(P(w) for w in words)

def product(nums):
    result = 1
    for x in nums:
        result *= x
    return result

In [31]:
tests = ['this is a test', 
         'this is a unusual test',
         'this is a neverbeforeseen test']

for test in tests:
    print(Pwords(tokens(test)), test)

2.9833963328e-11 this is a test
8.63747202302e-16 this is a unusual test
0.0 this is a neverbeforeseen test


References:

https://web.stanford.edu/class/cs276/pa/pa2.p

http://web.stanford.edu/~jurafsky/slp3/5.pdf

http://norvig.com/ngrams/spell-errors.txt

http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb

http://norvig.com/big.txt

http://norvig.com/ngrams/count_1w.txt

http://norvig.com/ngrams/